In [ ]:
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Model

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
import numpy as np

x_train = np.expand_dims(x_train, axis=-1)

x_train = np.repeat(x_train, 3, axis=-1)


In [ ]:

x_test = np.expand_dims(x_test, axis=-1)

x_test = np.repeat(x_test, 3, axis=-1)

In [ ]:
x_test.shape

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
model.fit(x_train, y_train, epochs=3, batch_size=32)

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)

print('Test accuracy:', test_acc)

In [ ]:
import matplotlib.pyplot as plt
import time

start_time = time.time()

history = model.fit(x_train, y_train, epochs=3)

end_time = time.time()

print("Total training time:", end_time - start_time, "seconds")


In [ ]:
start_time = time.time()

loss, test_acc = model.evaluate(x_test, y_test)

end_time = time.time()

print("Total testing time:", end_time - start_time, "seconds")


In [ ]:

plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.title('Accuracy vs. Loss')
plt.ylabel('Percentage')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()
